In [ ]:
### Classification Step - Amey + Rory

# Notes from Amey
"""
 - Lets ignore the validation data that they provided for now, just run a train-val-test
   split on a subset of the train data (pref train13.zip until we know that works, smallest)
 - Steps of ML pipeline:
    - Dataset
    - Load model
    - Loss calculation
    - training loop

"""

'\n - Lets ignore the validation data that they provided for now, just run a train-val-test\n   split on a subset of the train data (pref train13.zip until we know that works, smallest)\n - Steps of ML pipeline:\n    - Dataset\n    - Load model\n    - Loss calculation\n    - training loop\n\n'

In [ ]:
# pytorch lightning abstracts the training modules
# we can initialize a trainer module and avoid the need
!pip install pytorch-lightning

In [ ]:
!pip show pytorch-lightning

Name: pytorch-lightning
Version: 2.4.0
Summary: PyTorch Lightning is the lightweight PyTorch wrapper for ML researchers. Scale your models. Write less boilerplate.
Home-page: https://github.com/Lightning-AI/lightning
Author: Lightning AI et al.
Author-email: pytorch@lightning.ai
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: fsspec, lightning-utilities, packaging, PyYAML, torch, torchmetrics, tqdm, typing-extensions
Required-by: 


In [ ]:
import pandas as pd
import numpy as np
import torchvision
from torch.utils.data import Dataset
import os
from pathlib import Path
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torch
import torch.nn as nn
from pytorch_lightning import seed_everything

from torchmetrics.classification import Accuracy
from torchvision.io import read_image
seed_everything(42, workers=True)


INFO:lightning_fabric.utilities.seed:Seed set to 42


42

In [ ]:
# download subset of the SewerML data
!curl --location-trusted -u username:SewerML4294lsdaw321 -O "https://sciencedata.dk/public/Large%20AAU%20files/Sewer_ML/train13.zip"
# !curl --location-trusted -u username:SewerML4294lsdaw321 -O "https://sciencedata.dk/public/Large%20AAU%20files/Sewer_ML/valid00.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0 5978M    0 1088k    0     0   278k      0  6:06:13  0:00:03  6:06:10  278k^C


In [ ]:
!unzip -q train13.zip -d train13
# !unzip valid00.zip

[train13.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of train13.zip or
        train13.zip.zip, and cannot find train13.zip.ZIP, period.


In [ ]:
#Download labels
!curl --location-trusted -u username:SewerML4294lsdaw321 -O "https://sciencedata.dk/public/Large%20AAU%20files/Sewer_ML/SewerML_Train.csv"
#!curl --location-trusted -u username:SewerML4294lsdaw321 -O "https://sciencedata.dk/public/Large%20AAU%20files/Sewer_ML/SewerML_Val.csv"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55.9M  100 55.9M    0     0  2448k      0  0:00:23  0:00:23 --:--:-- 3181k


In [ ]:
train_labels = pd.read_csv("SewerML_Train.csv")
# val_labels = pd.read_csv("SewerML_Val.csv")

In [ ]:
#Define dataset and dataloader
class SewerMLDataset(Dataset):
  def __init__(self, data_root: Path, label_path: Path, split = "Train", transform = None):
    self.data_root = data_root
    self.file_list = os.listdir(data_root)[:256]
    self.label_df = pd.read_csv(label_path)
    self.transform = transform

  def __len__(self):
    return len(self.file_list)

  def __getitem__(self, idx):     # imo we should do the resizing and stuff here
    file_name = self.file_list[idx]
    file_path = os.path.join(self.data_root, file_name)
    img = read_image(file_path)

    if self.transform:
      img = self.transform(img)

    label = self.label_df[self.label_df["Filename"] == file_name]["Defect"].item()

    return img, label



In [ ]:
class ResNetModel(pl.LightningModule):
    def __init__(self, resnet_model):
        super().__init__()
        self.model = resnet_model
        num_ftrs = self.model.fc.in_features

        self.model.fc = nn.Linear(num_ftrs, 2)
        for param in self.model.parameters():
            param.requires_grad = False

        for param in self.model.fc.parameters():
            param.requires_grad = True

    def forward(self, x):
        x = self.model(x)
        # x = self.classifier(x)
        return x

In [ ]:
class SewerMLClassifier(pl.LightningModule):   # now generic; can use with any candidate model
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.loss = nn.CrossEntropyLoss() # don't need softmax layer in model
        self.accuracy = Accuracy(task="binary")


    def training_step(self, batch, batch_idx):
        inputs, target = batch
        output = self.model(inputs)
        loss = self.loss(output, target.view(-1))
        self.log('train_loss', loss, on_step=True, on_epoch=False, prog_bar=True, logger=True)
        preds = torch.argmax(output, dim=1)
        # print('train_acc_step: ', self.accuracy(preds, target))
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, target = batch
        output = self.model(inputs)
        loss = self.loss(output, target.view(-1))
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.1)

In [ ]:
#Define model
from torchvision.models import resnet18, ResNet18_Weights
resnet_model = resnet18(pretrained=True)

# print(resnet_model) # params check

# Initialize the Weight Transforms
# imo needs to go in dataset/dataloader
weights = ResNet18_Weights.DEFAULT
preprocess = weights.transforms()

# Apply it to the input image
# img_transformed = preprocess(img)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
print(preprocess)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [ ]:
train_path = Path("train13")
label_path = Path("SewerML_Train.csv")

train_dataset = SewerMLDataset(data_root=train_path, label_path=label_path, transform=preprocess)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
print(os.listdir(train_path)[0])

00235331.png


In [ ]:
#Define pytorch lihgtning trainer
trainer = pl.Trainer(max_epochs=5, log_every_n_steps=1)

model = SewerMLClassifier(ResNetModel(resnet_model))

trainer.fit(model, train_dataloader)

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type             | Params | Mode 
------------------

Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
trainer.test(model, train_dataloader)
